In [ ]:
#Notebook description

# This notebook evaluates the performance of a portfolio of assets on a buy-and-hold basis.
# It focuses on how multiple assets interact within the portfolio, rather than assessing a specific mechanical trading strategy.
# buy and hold is a good benchmark for any trading strategy, so it is important to evaluate the performance of a portfolio independently of 
# trading strategies we may want to implement on the individual assets.


In [ ]:
# LOAD DATA
import numpy as np
import pandas as pd

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from IPython.display import display

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")
import yfinance as yf
from Quantapp.Computation import Computation
from Quantapp.EconomicData import EconomicData


qc = Computation()
qe = EconomicData()



time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200

period = '10y'
interval = '1d'

#this is the portfolio and their coreesponding weights


def create_equal_weighted_dict(tickers):
    return {ticker: 1/len(tickers) for ticker in tickers}

class Portfolio:
    def __init__(self, ticker_dict ,period='10y', interval='1d', equal_weighted=False, portfolio_name='Portfolio'):
        self.portfolio_name = portfolio_name
        self.period = period
        self.interval = interval
        if equal_weighted:
            self.ticker_dict = create_equal_weighted_dict(list(ticker_dict.keys()))
        else:
            self.ticker_dict = ticker_dict
        self.tickers = list(ticker_dict.keys())
        self.data = self.get_data() #close prices of the tickers
        
    def __name__(self):
        return self.portfolio_name
    
    def create_equal_weighted_dict(self):
        return {ticker: 1/len(self.tickers) for ticker in self.tickers}
    
    def rebalance_to_equal_weighted(self):
        self.ticker_dict = self.create_equal_weighted_dict()
        
    #retrieve raw prices using yf.Ticker as a dataframe
    def retrieve_raw_prices(self, data_type='dataframe'):
        """
        Retrieves historical Close price data for all tickers in ticker_dict using yfinance.

        Returns:
            pd.DataFrame: A DataFrame containing Close prices with dates as index and tickers as columns.
        """
        # Download historical data for all tickers
        data = yf.download(
            tickers=self.tickers,
            period=self.period,
            interval=self.interval,
            auto_adjust=True,
            threads=True,
            progress=False
        )
        
        if data_type == 'dataframe':
            return data['Close']
        elif data_type == 'dict':
            return {ticker: data['Close'][ticker] for ticker in self.tickers}
        else:
            raise ValueError('data_type should be either dataframe or dict')

    def get_data(self):
        """
        Retrieves historical Close price data for all tickers in ticker_dict using yfinance.

        Returns:
            pd.DataFrame: A DataFrame containing Close prices with dates as index and tickers as columns.
        """
        # Download historical data for all tickers
        data = yf.download(
            tickers=self.tickers,
            period=self.period,
            interval=self.interval,
            auto_adjust=True,
            threads=True,
            progress=False
        )
        
        return data['Close']

    def returns(self, n=1, return_type='portfolio'):
        """
        Calculates the daily returns of the portfolio or individual assets.

        Args:
            n (int): The number of periods over which to calculate returns.
            allocation_type (str): Type of returns to calculate.
                                    - 'portfolio' (default): Returns the portfolio's daily returns.
                                    - 'individual': Returns a DataFrame of each individual asset's daily returns.

        Returns:
            pd.Series or pd.DataFrame: 
                - If allocation_type is 'portfolio', returns a Series of portfolio returns.
                - If allocation_type is 'individual', returns a DataFrame of individual asset returns.
        """
        # Calculate daily percentage change
        daily_returns = self.data.pct_change(n).dropna(how='all')

        if return_type == 'portfolio':
            # Compute portfolio returns by multiplying asset returns with their respective weights
            # and summing them up
            portfolio_returns = daily_returns.dot(list(self.ticker_dict.values()))
            return portfolio_returns
        elif return_type == 'individual':
            # Return individual asset returns as a DataFrame
            return daily_returns
        else:
            raise ValueError("Invalid return_type. Choose 'portfolio' or 'individual'.")
           
    def cumulative_returns(self, n=200, return_type='portfolio'):
        """
        Calculates the cumulative returns of the portfolio or individual assets over the last n days.

        Args:
            n (int): Number of recent days to calculate cumulative returns.
            allocation_type (str): Type of returns to calculate.
                                    - 'portfolio' (default): Returns cumulative returns of the portfolio.
                                    - 'individual': Returns cumulative returns of individual assets.

        Returns:
            pd.Series or pd.DataFrame: 
                - If 'portfolio', returns a Series of cumulative returns.
                - If 'individual', returns a DataFrame of cumulative returns for each asset.
        """

        # Retrieve the returns based on allocation type
        all_returns = self.returns(n=n, return_type=return_type)
        
        # Ensure there are enough data points
        if return_type == 'portfolio':
            if len(all_returns) < n:
                raise ValueError(f"Not enough data to calculate cumulative returns for the last {n} days.")
        elif return_type == 'individual':
            if len(all_returns) < n:
                raise ValueError(f"Not enough data to calculate cumulative returns for the last {n} days.")
        else:
            raise ValueError("Invalid return_type. Choose 'portfolio' or 'individual'.")
        
        # Slice the last n days of returns
        recent_returns = all_returns.iloc[-n:]
        
        if return_type == 'portfolio':
            # Calculate cumulative returns for the portfolio
            cumulative_returns = (1 + recent_returns).cumprod() - 1
            return cumulative_returns
        elif return_type == 'individual':
            # Calculate cumulative returns for each individual asset
            cumulative_returns = (1 + recent_returns).cumprod() - 1
            return cumulative_returns
    
    def rolling_sortino_ratio(self, n=21):
        """
        Calculates the rolling Sortino ratio of the portfolio.

        Args:
            n (int): The number of periods to consider in each rolling window.

        Returns:
            pd.Series: The rolling Sortino ratio of the portfolio.
        """
        # Calculate the daily returns of the portfolio
        returns = self.returns()
        
        # Calculate the daily downside deviation
        downside_deviation = returns.rolling(window=n).apply(lambda x: x[x < 0].std())
        
        # Calculate the rolling Sortino ratio
        rolling_sortino = np.sqrt(n) * returns.rolling(window=n).mean() / downside_deviation
        
        return rolling_sortino


    def rolling_correlation(self, benchmark, n=21):
        """
        Calculates the rolling correlation between the portfolio and a benchmark.

        Args:
            benchmark (pd.Series): The returns of the benchmark.
            n (int): The number of periods to consider in each rolling window.

        Returns:
            pd.Series: The rolling correlation between the portfolio and the benchmark.
        """
        # Calculate the daily returns of the benchmark
        benchmark_returns = benchmark
        
        # Calculate the rolling correlation
        rolling_correlation = self.returns().rolling(window=n).corr(benchmark_returns)
        
        return rolling_correlation
    
    def standard_deviation(self):
        """
        Calculates the standard deviation of the portfolio returns.

        Returns:
            float: The standard deviation of the portfolio returns.
        """
        # Calculate the standard deviation of the portfolio returns
        return self.returns().std()
    
    def benchmark_returns_minus_portfolio_returns(self, benchmark_ticker, n = 1):
        """
        Calculates the difference between the returns of the benchmark and the portfolio.

        Args:
            benchmark (pd.Series): The returns of the benchmark.

        Returns:
            pd.Series: The difference between the returns of the benchmark and the portfolio.
        """
        
        benchmark = yf.Ticker(benchmark_ticker).history(period=self.period, interval=self.interval)['Close'].pct_change(n).dropna()
        #convert benchmark index to portfolio index
        benchmark.index = self.returns(n).index
        # Calculate the difference between the benchmark and the portfolio returns
        excess_returns = benchmark - self.returns(n)
        
        return excess_returns
    
    def plot_rolling_sortino(self, plot_assets=False, n=21):
        """
        Plots the rolling Sortino ratio of the portfolio.
        If plot_assets is True, it will also plot the rolling Sortino ratio of each asset in the portfolio.

        Args:
            plot_assets (bool): Whether to plot the rolling Sortino ratio of each asset in the portfolio.
            n (int): The number of periods to consider in each rolling window.
        """

        import plotly.graph_objects as go
        import numpy as np

        # Calculate the rolling Sortino ratio of the portfolio
        portfolio_sortino = self.rolling_sortino_ratio(n=n)

        # Compute mean and standard deviation of the portfolio's Sortino ratio
        mean_val = portfolio_sortino.mean()
        std_val = portfolio_sortino.std()

        # Create the Plotly figure
        fig = go.Figure()
        
        # Add trace for portfolio Sortino ratio as a dashed line
        fig.add_trace(go.Scatter(
            x=portfolio_sortino.index,
            y=portfolio_sortino,
            mode='lines',
            name=f'{self.portfolio_name} Sortino',
            line=dict(width=4, dash='dash')
        ))

        # Define standard deviation levels and corresponding colors for shading
        std_levels = {
            1: {"color": "yellow", "opacity": 0.5},
            2: {"color": "LightCoral", "opacity": 0.5},
        }

        # Add mean and standard deviation lines using a loop to avoid redundancy
        for i in range(-3, 4):
            if i == 0:
                # Add mean line
                fig.add_hline(
                    y=mean_val,
                    line_dash="dash",
                    line_color="Green",
                    line_width=2,
                    annotation_text="Mean",
                    annotation_position="top left"
                )
            else:
                # Add lines for +/-1σ, +/-2σ, +/-3σ
                fig.add_hline(
                    y=mean_val + i * std_val,
                    line_dash="dash",
                    line_color="Red",
                    line_width=2,
                    annotation_text=f"{'+' if i > 0 else ''}{i}σ",
                    annotation_position="top left"
                )

        # Add shaded regions between standard deviation levels
        for i in std_levels:
            # Shaded region between +iσ and +(i+1)σ
            fig.add_shape(
                type="rect",
                x0=portfolio_sortino.index.min(),
                x1=portfolio_sortino.index.max(),
                y0=mean_val + i * std_val,
                y1=mean_val + (i + 1) * std_val,
                line=dict(color="Red", width=2, dash="dash"),
                fillcolor=std_levels[i]["color"],
                opacity=std_levels[i]["opacity"]
            )
            # Shaded region between - (i+1)σ and -iσ
            fig.add_shape(
                type="rect",
                x0=portfolio_sortino.index.min(),
                x1=portfolio_sortino.index.max(),
                y0=mean_val - (i + 1) * std_val,
                y1=mean_val - i * std_val,
                line=dict(color="Red", width=2, dash="dash"),
                fillcolor=std_levels[i]["color"],
                opacity=std_levels[i]["opacity"]
            )
            
        # Add annotation to the center of the plot  
        fig.add_annotation(
            x=portfolio_sortino.index[len(portfolio_sortino) // 2],
            y=mean_val,
            text=f'Portfolio Mean: {mean_val:.2f}',
            showarrow=False,
            yshift=10
        )

        # Add annotations for each shaded region
        for i in std_levels:
            fig.add_annotation(
            x=portfolio_sortino.index[len(portfolio_sortino) // 2],
            y=mean_val + (i + 0.5) * std_val,
            text=f'Portfolio +{i}σ to +{i+1}σ',
            showarrow=False,
            yshift=10
            )
            fig.add_annotation(
            x=portfolio_sortino.index[len(portfolio_sortino) // 2],
            y=mean_val - (i + 0.5) * std_val,
            text=f'Portfolio -{i+1}σ to -{i}σ',
            showarrow=False,
            yshift=10
            )
        
            

        # Optionally add traces for each asset's Sortino ratio as solid lines
        if plot_assets:
            for ticker in self.tickers:
                # Calculate asset returns
                asset_returns = self.data[ticker].pct_change().dropna()

                # Calculate downside deviation for rolling window
                downside_deviation = asset_returns.rolling(window=n).apply(lambda x: x[x < 0].std())

                # Calculate rolling Sortino ratio
                rolling_sortino = np.sqrt(n) * asset_returns.rolling(window=n).mean() / downside_deviation

                # Add asset Sortino ratio trace
                fig.add_trace(go.Scatter(
                    x=rolling_sortino.index,
                    y=rolling_sortino,
                    mode='lines',
                    name=f'{ticker} Sortino',
                    line=dict(width=2)
                ))

        # Add a horizontal line at zero
        fig.add_hline(
            y=0,
            line=dict(color="Red", width=2)
        )
        
        # Create a dropdown menu, the values should be 1 year, 3 years, 5 years, 10 years. Default value should be 3 years
        # Create a dropdown menu, the values should be 1 year, 3 years, 5 years, 10 years. Default value should be 3 years
        fig.update_layout(
            updatemenus=[
            dict(
            buttons=[
            dict(
                args=[{"xaxis.range": [portfolio_sortino.index[-252], portfolio_sortino.index[-1]]}],
                label="1 Year",
                method="relayout"
            ),
            dict(
                args=[{"xaxis.range": [portfolio_sortino.index[-756], portfolio_sortino.index[-1]]}],
                label="3 Years",
                method="relayout"
            ),
            dict(
                args=[{"xaxis.range": [portfolio_sortino.index[-1260], portfolio_sortino.index[-1]]}],
                label="5 Years",
                method="relayout"
            ),
            dict(
                args=[{"xaxis.range": [portfolio_sortino.index[0], portfolio_sortino.index[-1]]}],
                label="10 Years",
                method="relayout"
            )
            ],
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.25,
            yanchor="top",
            active=1  # Set default to 3 years
            ),
            ]
        )

        # Set the default view to 3 years
        fig.update_xaxes(range=[portfolio_sortino.index[-756], portfolio_sortino.index[-1]])

        
        # Update the layout of the plot for better readability and aesthetics
        fig.update_layout(
            title=f'Rolling Sortino Ratio of {self.portfolio_name}',
            xaxis_title='Date',
            yaxis_title='Rolling Sortino Ratio',
            legend_title='Ticker',
            template='plotly_dark',
            height=1000
        )

        # Display the plot
        fig.show()


market_portfolio = Portfolio({'SPY': 1},portfolio_name='Market Portfolio')
personal_portfolio = Portfolio({'XLV': .1,'MCHP': .1,'EWW': .1,'XLB': .1, 'FCX': .1, 'MRK': .1, 'XLB': .1, 'URA':.1,'XME':.1,'INDA':.1}, equal_weighted=True, portfolio_name='Personal Portfolio')


all_weather_portfolio = Portfolio({'SPY': 0.3, 'TLT': 0.4, 'GLD': 0.15, 'DBC': 0.15},portfolio_name='All Weather Portfolio')
three_fund_portfolio = Portfolio({'VTI': 0.6, 'VXUS': 0.3, 'BND': 0.1},portfolio_name='Three Fund Portfolio')
golden_butterfly_portfolio = Portfolio({'SPY': 0.2, 'TLT': 0.2, 'GLD': 0.2, 'SHY': 0.2, 'IWN': 0.2},portfolio_name='Golden Butterfly Portfolio')
risk_parity_portfolio = Portfolio({'SPY': 0.25, 'TLT': 0.25, 'GLD': 0.25, 'DBC': 0.25},portfolio_name='Risk Parity Portfolio')
sixty_forty_portfolio = Portfolio({'SPY': 0.6, 'TLT': 0.4},portfolio_name='Sixty Forty Portfolio')
core_four_portfolio = Portfolio({'VTI': 0.4, 'VXUS': 0.2, 'BND': 0.4},portfolio_name='Core Four Portfolio')
coffeehouse_portfolio = Portfolio({'VTI': 0.4, 'VXUS': 0.15, 'BND': 0.15, 'GLD': 0.15, 'VNQ': 0.15},portfolio_name='Coffeehouse Portfolio')
all_sectors_portfolio = Portfolio({'XLC': 0.1, 'XLY': 0.1, 'XLP': 0.1, 'XLE': 0.1, 'XLF': 0.1, 'XLV': 0.1, 'XLI': 0.1, 'XLB': 0.1, 'XLRE': 0.1, 'XLK': 0.1, 'XLU': 0.1, 'SPY': 0.1}, portfolio_name='All Sectors Portfolio')
spy_vix_portfolio = Portfolio({'SPY': 0.5, '^VIX': 0.5},portfolio_name='SPY VIX Portfolio')

factor_portfolio = Portfolio({'SIZE': 0.2, 'MTUM': 0.2,'VLUE': 0.2,'QUAL': 0.2,'USMV': 0.2},portfolio_name='Factor Portfolio')
multi_factor_portfolio = Portfolio({'LRGF':0.25,'INTF':0.25,'GLOF':0.5}, portfolio_name='Multifactor Portfolio')
minimum_volatility_factor_portfolio = Portfolio({'USMV': 0.5, 'EFAV': 0.25, 'EEMV': 0.25},portfolio_name='Minimum Volatility Portfolio')
all_factors_portfolio = Portfolio({'SIZE': 0.2, 'MTUM': 0.2,'VLUE': 0.2,'QUAL': 0.2,'USMV': 0.2,'LRGF':0.2,'INTF':0.2,'GLOF':0.2,'EFAV': 0.2, 'EEMV': 0.2},portfolio_name='All Factors Portfolio')

selected_portfolio = personal_portfolio
selected_portfolio.rebalance_to_equal_weighted()



In [ ]:
#Plot rolling Sortino ratios of assets inlcuding the portfolio as a whole
selected_portfolio.plot_rolling_sortino(plot_assets=True, n=200)


In [ ]:
#this tells you to increase capital allocation to the asset. this does not tell you whether or not its a good investment
from scipy.optimize import minimize

#columns are the tickers and rows are the dates
dataframe = selected_portfolio.retrieve_raw_prices()
def optimal_allocation(dataframe, risk_free_rate=0.0):
    """
    Calculates the optimal allocation of assets in a portfolio to maximize the Sharpe ratio
    using mean-variance optimization.

    :param dataframe: A DataFrame of asset prices (columns=tickers, rows=dates).
    :param risk_free_rate: The risk-free rate for Sharpe ratio calculation.
    :return: A DataFrame with one column 'weight' and rows as asset names, rounded to 2 decimals.
    """
    import numpy as np
    import pandas as pd
    from scipy.optimize import minimize

    # 1) Compute daily returns from price data
    returns_df = dataframe.pct_change().dropna()

    # 2) Calculate average (annualized) returns (252: trading days/year)
    mean_returns = returns_df.mean() * 252

    # 3) Calculate the annualized covariance matrix
    cov_matrix = returns_df.cov() * 252

    # 4) Define the number of assets
    num_assets = len(dataframe.columns)

    # 5) Objective function to maximize the Sharpe ratio => minimize negative Sharpe
    def neg_sharpe(weights):
        ret = np.dot(weights, mean_returns)
        vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -(ret - risk_free_rate) / vol

    # 6) Constraint: sum(weights) = 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

    # 7) Bounds: no short selling => each weight in [0,1]
    bounds = tuple((0.0, 1.0) for _ in range(num_assets))

    # 8) Initial guess: equal allocation
    init_guess = [1.0 / num_assets] * num_assets

    # 9) Optimization
    result = minimize(
        neg_sharpe,
        init_guess,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )

    # 10) Return a DataFrame of weights, rounded to two decimals
    weights_df = pd.DataFrame(result.x, index=dataframe.columns, columns=['weight'])
    weights_df = weights_df.round(2)
    return weights_df

def compute_rolling_sortino_ratios(dataframe, window=21):
    
    # Calculate daily returns
    returns = dataframe.pct_change().dropna()
    
    # Calculate downside deviation for each rolling window
    downside_deviation = returns.rolling(window=window).apply(lambda x: x[x < 0].std())
    
    # Calculate the rolling Sortino ratio
    rolling_sortino = np.sqrt(window) * returns.rolling(window=window).mean() / downside_deviation
    
    return rolling_sortino

def categorize_sortino_ratios(dataframe, window=21):
    """
    Categorizes rolling Sortino ratios based on their z-scores, accounting for both positive and negative deviations.

    Args:
        dataframe (pd.DataFrame): DataFrame containing the Sortino ratios.
                                  Columns represent different assets or portfolio,
                                  and rows represent different dates.
        window (int): The number of periods to consider in each rolling window.

    Returns:
        pd.DataFrame: DataFrame with the same structure as `dataframe`,
                      containing category labels:
                      - -3: Sortino ratio beyond -3 standard deviations from the mean.
                      - -2: Sortino ratio between -3 and -2 standard deviations below the mean.
                      - -1: Sortino ratio between -2 and -1 standard deviations below the mean.
                      - 0: Sortino ratio within ±1 standard deviation of the mean.
                      - 1: Sortino ratio between +1 and +2 standard deviations above the mean.
                      - 2: Sortino ratio between +2 and +3 standard deviations above the mean.
                      - 3: Sortino ratio beyond +3 standard deviations above the mean.
    """
    import numpy as np
    import pandas as pd

    # Compute the rolling Sortino ratios
    rolling_sortino = compute_rolling_sortino_ratios(dataframe, window=window)

    # Compute the rolling mean and standard deviation for each asset
    rolling_mean = rolling_sortino.rolling(window=window).mean()
    rolling_std = rolling_sortino.rolling(window=window).std()

    # Compute the z-scores for each Sortino ratio
    z_scores = (rolling_sortino - rolling_mean) / rolling_std

    # Define a function to categorize based on z-score with direction
    def categorize(z):
        if z <= -3:
            return -3
        elif -3 < z <= -2:
            return -2
        elif -2 < z <= -1:
            return -1
        elif -1 < z <= 1:
            return 0
        elif 1 < z <= 2:
            return 1
        elif 2 < z <= 3:
            return 2
        else:  # z > 3
            return 3

    # Apply the categorization function to each element in the z_scores DataFrame
    categories = z_scores.applymap(categorize)

    return categories

def plot_sortino_ratios(dataframe, window=21):
    """
    Plots the rolling Sortino ratios for each asset in the given DataFrame and
    indicates when each Sortino ratio enters the 2-3 standard deviation zone
    (i.e., categories -3, -2, 2, or 3) as defined by the categorize_sortino_ratios function.

    Args:
        dataframe (pd.DataFrame): A DataFrame of raw price data from which Sortino ratios will be computed.
        window (int): The rolling window used to compute Sortino ratios and categories.

    Returns:
        None
    """
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go

    # 1. Compute the rolling Sortino ratios
    rolling_sortino = compute_rolling_sortino_ratios(dataframe, window=window)

    # 2. Categorize Sortino ratios
    categories = categorize_sortino_ratios(dataframe, window=window)

    # 3. Create a Plotly figure
    fig = go.Figure()

    # 4. Add a trace for each asset's rolling Sortino ratio
    for col in rolling_sortino.columns:
        fig.add_trace(
            go.Scatter(
                x=rolling_sortino.index,
                y=rolling_sortino[col],
                mode='lines',
                name=f"{col} Sortino Ratio"
            )
        )

    # 5. Overlay markers only when the Sortino ratio enters the 2-3 standard deviation zone
    #    (i.e., when category changes from something else into {-3, -2, 2, 3}).
    sd_zone_set = {-3, -2, 2, 3}
    for col_index, col in enumerate(rolling_sortino.columns):
        x_arrows = []
        y_arrows = []
        marker_symbols = []
        trace_color = "rgba(255,255,255,0.8)"  # Marker color; adjust as needed

        ratio_series = rolling_sortino[col]
        cat_series = categories[col]

        for i in range(1, len(ratio_series)):
            idx_current = ratio_series.index[i]
            idx_previous = ratio_series.index[i - 1]

            # If categories are NaN, skip
            if pd.isna(cat_series.loc[idx_current]) or pd.isna(cat_series.loc[idx_previous]):
                continue

            cat_current = cat_series.loc[idx_current]
            cat_previous = cat_series.loc[idx_previous]

            # Check if we have just entered the 2-3 std zone
            if (cat_previous not in sd_zone_set) and (cat_current in sd_zone_set):
                x_arrows.append(idx_current)
                y_arrows.append(ratio_series.loc[idx_current])

                # Arrow-up if category is increasing, arrow-down otherwise
                if cat_current > cat_previous:
                    marker_symbols.append("arrow-up")
                else:
                    marker_symbols.append("arrow-down")

        # Add scatter trace for the markers
        if x_arrows:
            fig.add_trace(
                go.Scatter(
                    x=x_arrows,
                    y=y_arrows,
                    mode='markers',
                    marker=dict(
                        symbol=marker_symbols,
                        size=12,
                        color=trace_color,
                        line=dict(width=1, color="black")
                    ),
                    name=f"{col} enters ±2-3 SD",
                    showlegend=False
                )
            )

    # 6. Add layout, title, and axis labels
    fig.update_layout(
        title='Rolling Sortino Ratios with SD Zone Entries',
        xaxis_title='Date',
        yaxis_title='Sortino Ratio',
        template='plotly_dark',
        height=600
    )

    # 7. Display the plot
    fig.show()
    
def rolling_optimal_allocation(dataframe, window=50, risk_free_rate=0.0):
    """
    Applies the 'optimal_allocation' function over a rolling window.
    Returns a DataFrame where each row corresponds to the weights 
    computed for that window, indexed by the last date in the window.
    """
    import pandas as pd

    # Compute daily returns (drop the first NaN row)
    returns_df = dataframe.pct_change().dropna()

    # List to store each window's optimal weights
    rolling_weights_list = []

    # We only compute weights once we have 'window' days of data
    for i in range(window, len(returns_df) + 1):
        # Price slice for the current rolling window
        slice_data = dataframe.iloc[i - window : i]

        # Reuse the existing 'optimal_allocation' function
        weights_df = optimal_allocation(slice_data, risk_free_rate=risk_free_rate)

        # Append just the weight values (in the same column order as 'dataframe') to the list
        rolling_weights_list.append(weights_df['weight'].values)

    # Create a DataFrame of rolling weights, indexed by the final date in each window
    rolling_index = returns_df.index[window - 1 :]
    rolling_weights_df = pd.DataFrame(
        rolling_weights_list,
        index=rolling_index,
        columns=dataframe.columns
    )

    return rolling_weights_df

def plot_rolling_portfolio_allocation(results):
    """
    Plots the rolling optimal portfolio allocation with annotations for the latest weights.

    Args:
        results (pd.DataFrame): DataFrame containing the portfolio allocation weights.

    Returns:
        None
    """
    import plotly.graph_objects as go

    # Create the Plotly figure
    fig = go.Figure()

    # Add a trace for each asset's weight
    for col in results.columns:
        fig.add_trace(go.Scatter(
            x=results.index,
            y=results[col],
            mode='lines',
            name=col
        ))

    # Add annotations for the latest weight of each asset
    for col in results.columns:
        latest_date = results.index[-1]
        latest_weight = results[col].iloc[-1]
        fig.add_annotation(
            x=latest_date,
            y=latest_weight,
            text=f"{latest_weight:.2f}",
            xanchor='left',
            yanchor='middle',
            showarrow=False,
            font=dict(color=fig.data[results.columns.get_loc(col)].line.color),
            bgcolor="rgba(255,255,255,0.5)"
        )

    # Update layout with title, axis labels, and template
    fig.update_layout(
        title='Rolling Optimal Portfolio Allocation',
        xaxis_title='Date',
        yaxis_title='Weight',
        template='plotly_dark',
        height=600
    )

    # Adjust layout to make space for annotations on the right
    fig.update_layout(
        margin=dict(r=150)
    )

    # Display the plot
    fig.show()



raw_prices = selected_portfolio.retrieve_raw_prices()
results = rolling_optimal_allocation(dataframe,window=200,risk_free_rate=0.0)

plot_sortino_ratios(dataframe, window=200)
plot_rolling_portfolio_allocation(results)

'''
raw_prices = selected_portfolio.retrieve_raw_prices()
results = rolling_optimal_allocation(dataframe,window=50,risk_free_rate=0.0)

plot_sortino_ratios(dataframe, window=50)
plot_rolling_portfolio_allocation(results)
'''

In [ ]:
selected_portfolio.rolling_sortino_ratio(n=200)
